In [1]:
from collections import defaultdict
import copy
import random
import os
import shutil
import glob
from urllib.request import urlretrieve

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

cudnn.benchmark = True

In [2]:
# gpu?
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
traindata_path = glob.glob("C:/Users/rh987/.fastai/data/cifar10/train/*/*")
testdata_path = glob.glob("C:/Users/rh987/.fastai/data/cifar10/test/*/*")
traindata_path[:2]

['C:/Users/rh987/.fastai/data/cifar10/train\\airplane\\10008_airplane.png',
 'C:/Users/rh987/.fastai/data/cifar10/train\\airplane\\10010_airplane.png']

In [4]:
classes = [i.split("\\")[-1] for i in glob.glob("C:/Users/rh987/.fastai/data/cifar10/train/*")]
classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [5]:
from torch.utils.data import Dataset

# super init이 필요없다.

class CIFAR10Dataset(Dataset):
    def __init__(self, images_filepaths, transform=None):
        self.images_filepaths = images_filepaths
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = image_filepath.split("\\")[-2] # classname
        label = classes.index(label)
        if self.transform is not None:
            image = self.transform(img=image)
        return image, label

In [6]:
import torchvision.transforms as transforms
import torchvision

train_transform = transforms.Compose(
    [transforms.ToTensor(), # 0~1의 범위를 가지도록 정규화
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))] # -1~1의 범위를 가지도록 정규화 (standard)
)

train_dataset = CIFAR10Dataset(images_filepaths=traindata_path, transform=train_transform)

In [7]:
val_transform = transforms.Compose(
    [transforms.ToTensor(), # 0~1의 범위를 가지도록 정규화
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))] # -1~1의 범위를 가지도록 정규화 (standard)
)


val_dataset = CIFAR10Dataset(images_filepaths=testdata_path, transform=val_transform)

In [8]:
for image, label in train_dataset:
    print(image.size())
    print(type(label))
    break

torch.Size([3, 32, 32])
<class 'int'>


In [9]:
params = {
    "device": "cuda",
    "lr": 0.001,
    "batch_size": 64,
    "num_workers": 0,
    "epochs": 10,
}

In [10]:
train_loader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=0)

## VGG16 Network

In [11]:
import torch.nn as nn
import torch.nn.functional as F

from torchsummary import summary

# Conv2d class
# Conv2d(in_channels, out_channels, kernel_size,
#        stride=1, padding=0, dilation=1,groups=1,
#        bias=True, padding_mode='zeros', device=None, dtype=None)
# MaxPool2d(kernel_size, stride=None, padding=0)

class VGG16_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.vgg_block1 = nn.Sequential(
            nn.Conv2d(3,64,3,stride=1,padding=1),
            nn.Conv2d(64,64,3,stride=1,padding=1),
            nn.MaxPool2d(2,2)
        )
        self.vgg_block2 = nn.Sequential(
            nn.Conv2d(64,128,3,stride=1,padding=1),
            nn.Conv2d(128,128,3,stride=1,padding=1),
            nn.MaxPool2d(2,stride=2)
        )
        self.vgg_block3 = nn.Sequential(
            nn.Conv2d(128,256,3,stride=1,padding=1),
            nn.Conv2d(256,256,3,stride=1,padding=1),
            nn.Conv2d(256,256,3,stride=1,padding=1),
            nn.MaxPool2d(2,stride=2)
        )
        self.vgg_block4 = nn.Sequential(
            nn.Conv2d(256,512,3,stride=1,padding=1),
            nn.Conv2d(512,512,3,stride=1,padding=1),
            nn.Conv2d(512,512,3,stride=1,padding=1),
            nn.MaxPool2d(2,stride=2)
        )
        self.vgg_block5 = nn.Sequential(
            nn.Conv2d(512,512,3,stride=1,padding=1),
            nn.Conv2d(512,512,3,stride=1,padding=1),
            nn.Conv2d(512,512,3,stride=1,padding=1),
            nn.MaxPool2d(2,stride=2)
        )
        self.fc1 = nn.Linear(512,1024) # 너무 커서 대체
        self.fc2 = nn.Linear(1024,1024)
        self.fc3 = nn.Linear(1024,10)
        
    def forward(self, x):
        x = self.vgg_block1(x)
        x = self.vgg_block2(x)
        x = self.vgg_block3(x)
        x = self.vgg_block4(x)
        x = self.vgg_block5(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x
    
net = VGG16_Net().to(device)

# summary 는 device에 넣어야 볼 수 있다.
summary(net, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
            Conv2d-2           [-1, 64, 32, 32]          36,928
         MaxPool2d-3           [-1, 64, 16, 16]               0
            Conv2d-4          [-1, 128, 16, 16]          73,856
            Conv2d-5          [-1, 128, 16, 16]         147,584
         MaxPool2d-6            [-1, 128, 8, 8]               0
            Conv2d-7            [-1, 256, 8, 8]         295,168
            Conv2d-8            [-1, 256, 8, 8]         590,080
            Conv2d-9            [-1, 256, 8, 8]         590,080
        MaxPool2d-10            [-1, 256, 4, 4]               0
           Conv2d-11            [-1, 512, 4, 4]       1,180,160
           Conv2d-12            [-1, 512, 4, 4]       2,359,808
           Conv2d-13            [-1, 512, 4, 4]       2,359,808
        MaxPool2d-14            [-1, 51

In [12]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=params["lr"]) # optmizizer 안에 network의 parameters를 넣어둠

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # optimizer 안의 grad를 초기화
        optimizer.zero_grad()
        
        # forward
        outputs = net(inputs) # last activation 없음
        
        # loss
        loss = criterion(outputs, labels)
        
        # calc gard
        loss.backward()
        
        # backpropagation
        optimizer.step()
        
        # show
        running_loss += loss.item() # 로스값 추가
        if i % 200 == 199:
            print(f"[{epoch+1:2d}/{EPOCHS}, iter:{i+1:5d}], loss:{running_loss/200:.6f}")
            running_loss = 0.0
print("End")

[ 1/10, iter:  200], loss:94.916866
[ 1/10, iter:  400], loss:4.040245
[ 1/10, iter:  600], loss:3.662735


이게되네